## Regression of a function with a neural network in 1D

### Exercise 2:
Modify the code to solve the differential equation
$$
\frac{dy}{dx} = -y(x);\; y(0) = 1
$$
using
 a) finite difference method to calculate derivatives.
 b) automatic differentiation to calculate derivatives. 
Add boundary conditions as another loss function.

### Exercise 3:
Solve the same differential equation using
$$
y = 1+x\,\mathrm{Net}(x)
$$
This way we do not need to add another loss function.

### Exercise 4:
Solve the differential equation
$$
\frac{dy}{dx} = -y\,(x-1);\; y(0) = 1
$$
Run the code twice to see whether the results are the same. The results for two successive runs may not be the same. Explain why this is the case.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Define the function you want to fit
def true_function(x):
    return torch.sin(x)

# Generate synthetic data
x = torch.linspace(-5, 5, 100)
y = true_function(x)

# Define a simple neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the neural network
net = Net()

# Define the loss function
criterion = nn.MSELoss()

# Define the optimizer
optimizer = optim.LBFGS(net.parameters(), lr=0.1)

# Training loop
for i in range(100):
    def closure():
        optimizer.zero_grad()
        outputs = net(x.unsqueeze(-1))
        loss = criterion(outputs, y.unsqueeze(-1))
        loss.backward()
        return loss

    loss=optimizer.step(closure)
    if(i%10==0):
      print(i," ",loss.item())
# Print the final loss
with torch.no_grad():
    outputs = net(x.unsqueeze(-1))
    loss = criterion(outputs, y.unsqueeze(-1))
print(f'Final loss: {loss.item()}')
# Plot the results
plt.scatter(x.numpy(), y.numpy(), label='True Function', color='blue')
plt.plot(x.numpy(), outputs.numpy(), label='Neural Network Prediction', color='red')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()